In [113]:
import requests
import json
import pandas as pd

In [1]:
token = 'tobe'

In [7]:
moves_url = 'https://online.moysklad.ru/api/remap/1.2/entity/move'
headers_auth = {'Authorization': f'Basic {token}'}
moves_response = requests.get(moves_url, headers = headers_auth)

moves = moves_response.json().get('rows', [])

In [115]:
class MSMovePosition:
    def __init__(self, data:dict, token:str):
        self.auth_header = {'Authorization': f'Basic {token}'}
        self.quantity = self.get_quantity_from_data(data)
        self.position_info = self.get_position_info(data)
        
    @property
    def barcode(self):
        barcodes = self.position_info['barcodes']
        for barcode in barcodes:
            if 'ean13' in barcode:
                return barcode['ean13']
    @property
    def product_name(self):
        return self.position_info.get('name', None)
    
    @property
    def external_code(self):
        return self.position_info.get('externalCode')
        
    @staticmethod
    def get_quantity_from_data(data):
        return data.get('quantity', None)
    
    def get_position_info(self, data):
        pos_meta = data['meta']
        position_data_request_url = pos_meta.get('href', None)
        if position_data_request_url is None:
            return
        
        position_info = requests.get(position_data_request_url, headers=self.auth_header)
        position_info_dict = position_info.json()

        products_assortiment = position_info_dict.get('assortment', None)
        if products_assortiment is None:
            return dict()
        
        product_data_response = requests.get(products_assortiment['meta']['href'], headers=self.auth_header)
        return product_data_response.json()
    

class StoresManager:
    ALL_STORES_REQUEST_URL = 'https://online.moysklad.ru/api/remap/1.2/entity/store'
    def __init__(self, token):
        self.data = requests.get(self.ALL_STORES_REQUEST_URL, headers={'Authorization': f'Basic {token}'}).json()
        self.href_name_map = self.get_stores_name_map()
    
    def get_stores_name_map(self):
        result = dict()
        for store in self.data.get('rows', []):
            result[store['meta']['href']] = store['name']
        return result
    
    def get_store_name_by_href(self, store_href):
        return self.href_name_map.get(store_href, None)
        
class MSMove:
    MAIN_STORE_URL = 'https://online.moysklad.ru/api/remap/1.2/entity/store/82b96dcf-58a3-11eb-0a80-022e004085ff'
    BOX_STOTE_URL = 'https://online.moysklad.ru/api/remap/1.2/entity/store/be5071a7-61ae-11eb-0a80-06ae0001c060'
    
    def __init__(self, data:dict, token:str):
        self.stores_manager = StoresManager(token)
        self.token = token
        self.auth_header = {'Authorization': f'Basic {token}'}
        
        self.box_number = self.get_box_number_from_data(data)
        self.positions = self.get_position_list_by_data(data)
        self.source_store_href = data['sourceStore']['meta']['href']
        self.target_store_href = data['targetStore']['meta']['href']
        self.action_date = self.get_created_date_from_data(data)
        
    @property
    def target_store_name(self):
        return self.stores_manager.get_store_name_by_href(self.target_store_href)
    
    @property
    def source_store_name(self):
        return self.stores_manager.get_store_name_by_href(self.source_store_href)
    
    @staticmethod
    def get_created_date_from_data(data):
        date =  data.get('created', None)
        if date is not None:
            date = date.split(' ')[0]
        return date
    
    @staticmethod
    def get_source_name_from_data(data):
        store_request_url = data['sourceStore']['meta']['href']
        store_name = self.get_store_name_by_url(store_request_url)
        
    @staticmethod
    def get_box_number_from_data(data):
        attributes = data.get('attributes', [])
        for attr in attributes:
            if attr['name'] == 'Номер короба':
                return attr['value']
        return ''
    
    def get_position_list_by_data(self, data):
        result = []
        
        positions_meta = data['positions']['meta']
        positions_request_url = positions_meta['href']
        positions_data = requests.get(positions_request_url, headers=self.auth_header)
        positions_json = positions_data.json()
        if not positions_json.get('meta', False):
            return result
        if positions_json['meta']['size'] == 0:
            return result
        
        for position_data in positions_json['rows']:
            pos = MSMovePosition(position_data, self.token)
            result.append(pos)
            
        return result

In [116]:
df_data = []

stores_manager = StoresManager(token)
for move in moves:
    move = MSMove(move, token)
    data = dict()
    data['box_number'] = move.box_number
    data['source_store'] = move.source_store_name
    data['target_store'] = move.target_store_name
    data['action_date'] = move.action_date
    
    for position in move.positions:
        row_data = data
        row_data['barcode'] = position.barcode
        row_data['quantity'] = int(position.quantity)
        df_data.append(row_data.copy())

In [117]:
pd.DataFrame(df_data)

,box_number,source_store,target_store,action_date,barcode,quantity
0,2021-02-07_01,Основной склад,Готовые короба,2021-02-07,2000473874052,2
1,2021-02-07_01,Основной склад,Готовые короба,2021-02-07,4631147544117,4
2,2021-02-07_01,Основной склад,Готовые короба,2021-02-07,4631147544001,2
3,,Основной склад,Контейнер,2021-02-07,4631147544117,3
4,,Основной склад,Контейнер,2021-02-07,4631147544001,2
